- # Visualizando os Leads que preencheram formulário:

In [49]:
import os
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
!pip install psycopg2-binary python-dotenv -q

load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

def visualizar_leads_local():
    """
    Conecta ao banco de dados no Render e exibe os dados da tabela 'leads'
    em um formato de tabela (DataFrame).
    """
    if not DATABASE_URL:
        print("\nERRO: A variável DATABASE_URL não foi encontrada no seu arquivo .env.")
        print("Certifique-se de que o arquivo está na mesma pasta do seu notebook e que a variável está definida.")
        return

    conn = None
    try:
        import psycopg2 # Importa a biblioteca que acabamos de instalar
        
        print("\nConectando ao banco de dados PostgreSQL no Render...")
        conn = psycopg2.connect(DATABASE_URL)
        print("Conexão bem-sucedida!")

        # Comando SQL para selecionar todos os dados da tabela 'leads'
        sql_query = "SELECT * FROM leads ORDER BY data_ingestao DESC;"

        print("Buscando os leads cadastrados...")
        df = pd.read_sql_query(sql_query, conn)

        if df.empty:
            print("\nNenhum lead encontrado na tabela ainda.")
        else:
            print(f"\nExibindo {len(df)} leads encontrados:")
            display(df) # Exibe a tabela formatada no Jupyter

    except Exception as e:
        print(f"\nERRO: Não foi possível buscar os dados. Causa: {e}")

    finally:
        if conn:
            conn.close()
            print("\nConexão com o banco de dados foi fechada.")

if __name__ == '__main__':
    visualizar_leads_local()




[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Desktop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



Conectando ao banco de dados PostgreSQL no Render...
Conexão bem-sucedida!
Buscando os leads cadastrados...


C:\Users\Desktop\AppData\Local\Temp\ipykernel_8724\447959727.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)



Exibindo 26 leads encontrados:


,id,nome,email,telefone,cargo,empresa,nascimento,data_admissao,data_ingestao,status_n8n
0,29,Leandro Lead Quente,leandro.bambuvisual@gmail.com,1191334827,Diretor de Vendas,LeadFlow,None,None,2025-10-16 00:04:42.433782+00:00,PENDENTE
1,28,Pamela,pamela.bruninho28@gmail.com,1155624149,Professora,Oceano,None,None,2025-10-15 23:58:02.984194+00:00,PENDENTE
2,27,Leandro LeadFrio,leandroandrade020797@gmail.com,(11) 91332-4827,Vendedor,LeadFlow,None,None,2025-10-15 23:57:49.204121+00:00,PENDENTE
3,26,Leanttro ESTUDANTE,leandroandrade020797@gmail.com,(11)91332-4827,Estudante,Leandro,None,None,2025-10-15 23:55:34.678593+00:00,PENDENTE
4,25,LEANTTRO,leandroandrade020797@gmail.com,11963195328,PROFESSOR,lelis,None,None,2025-10-15 23:40:18.849207+00:00,PENDENTE
5,24,Leandro Andrade,leandroandrade020797@gmail.com,11985474101,teste,Teste,None,None,2025-10-15 22:05:17.013459+00:00,PENDENTE
6,23,Julio,leandro.bambuvisual@gmail.com,4545415,teste2,dfgggg,None,None,2025-10-15 21:47:22.001948+00:00,PENDENTE
7,22,Testenumero1000,leandroandrade020797@gmail.com,151515,teste,treeeer,None,None,2025-10-15 21:44:42.328689+00:00,PENDENTE
8,21,Leandro Andrade,leandro.bambuvisual@gmail.com,11913324827,Professor,Lelis,None,None,2025-10-15 21:37:06.355761+00:00,PENDENTE
9,20,LEANDRO ANDRADE DE OLIVEIRA,leandroandrade020797@gmail.com,teste,Diretor,teste,None,None,2025-10-15 20:52:50.429712+00:00,PENDENTE



Conexão com o banco de dados foi fechada.


- # ENVIAR LEAD PARA N8N CLASSIFICAR!

In [41]:
import os
import json
import requests
from dotenv import load_dotenv
!pip install psycopg2-binary python-dotenv requests -q

print("\nCarregando variáveis de ambiente (DATABASE_URL e N8N_WEBHOOK_URL)...")
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
N8N_WEBHOOK_URL = "https://leanttro.app.n8n.cloud/webhook-test/2b2fb1d4-b173-490a-9589-898dcfdbc855"

def enviar_leads_pendentes_para_n8n():
    """
    Busca leads com status 'PENDENTE' no banco de dados, envia para o n8n
    e atualiza o status para 'PROCESSADO' após o envio bem-sucedido.
    """
    # Validação das variáveis de ambiente
    if not DATABASE_URL or not N8N_WEBHOOK_URL:
        print("\nERRO: Verifique se as variáveis DATABASE_URL e N8N_WEBHOOK_URL estão definidas no seu arquivo .env.")
        return

    # Importa a biblioteca do DB aqui para garantir que foi instalada
    import psycopg2
    import psycopg2.extras

    conn = None
    try:
        print("\nConectando ao banco de dados PostgreSQL no Render...")
        conn = psycopg2.connect(DATABASE_URL)
        # Usamos DictCursor para obter os resultados como dicionários, o que facilita o envio
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        print("Conexão bem-sucedida!")

        # Busca apenas os leads que o n8n ainda não processou
        sql_query = "SELECT * FROM leads WHERE status_n8n = 'PENDENTE';"
        print("Buscando leads pendentes...")
        cur.execute(sql_query)
        leads_pendentes = cur.fetchall()

        if not leads_pendentes:
            print("\nNenhum lead novo para enviar ao n8n.")
            return

        print(f"\nEncontrados {len(leads_pendentes)} leads para enviar. Iniciando o processo...")
        leads_enviados_com_sucesso = 0

        for lead in leads_pendentes:
            lead_dict = dict(lead)
            lead_id = lead_dict['id']

            # Converte objetos de data/datetime para string no formato ISO, que é o padrão para JSON
            for key, value in lead_dict.items():
                if hasattr(value, 'isoformat'):
                    lead_dict[key] = value.isoformat()
            
            try:
                # Envia o lead para o webhook do n8n
                response = requests.post(N8N_WEBHOOK_URL, data=json.dumps(lead_dict), headers={'Content-Type': 'application/json'})
                
                if response.status_code == 200:
                    print(f"  - Lead ID {lead_id} enviado com sucesso para o n8n.")
                    
                    # Se o envio foi bem-sucedido, atualiza o status no banco de dados
                    update_sql = "UPDATE leads SET status_n8n = 'PROCESSADO' WHERE id = %s;"
                    cur.execute(update_sql, (lead_id,))
                    conn.commit() # Salva a alteração no banco
                    print(f"    -> Status do Lead ID {lead_id} atualizado para 'PROCESSADO'.")
                    leads_enviados_com_sucesso += 1
                else:
                    print(f"  - ERRO ao enviar lead ID {lead_id}. Status: {response.status_code}, Resposta: {response.text}")

            except requests.exceptions.RequestException as e:
                print(f"  - ERRO de conexão ao enviar lead ID {lead_id}: {e}")
        
        print(f"\nProcesso finalizado. {leads_enviados_com_sucesso} de {len(leads_pendentes)} leads foram enviados e atualizados.")

    except Exception as e:
        print(f"\nERRO GERAL: Ocorreu um problema durante a execução. Causa: {e}")
        if conn:
            conn.rollback() # Desfaz qualquer alteração pendente em caso de erro

    finally:
        if conn:
            cur.close()
            conn.close()
            print("\nConexão com o banco de dados foi fechada.")


# --- 4. EXECUTA A FUNÇÃO ---
if __name__ == '__main__':
    enviar_leads_pendentes_para_n8n()


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Desktop\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



Carregando variáveis de ambiente (DATABASE_URL e N8N_WEBHOOK_URL)...

Conectando ao banco de dados PostgreSQL no Render...
Conexão bem-sucedida!
Buscando leads pendentes...

Encontrados 2 leads para enviar. Iniciando o processo...
  - Lead ID 19 enviado com sucesso para o n8n.
    -> Status do Lead ID 19 atualizado para 'PROCESSADO'.
  - ERRO ao enviar lead ID 20. Status: 404, Resposta: {"code":404,"message":"The requested webhook \"2b2fb1d4-b173-490a-9589-898dcfdbc855\" is not registered.","hint":"Click the 'Execute workflow' button on the canvas, then try again. (In test mode, the webhook only works for one call after you click this button)"}

Processo finalizado. 1 de 2 leads foram enviados e atualizados.

Conexão com o banco de dados foi fechada.


In [48]:
import os
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display
import psycopg2

load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

def visualizar_leads_quentes():
    """
    Conecta ao banco de dados no Render e exibe os dados da tabela 'leads_quentes'
    em um formato de tabela (DataFrame).
    """
    if not DATABASE_URL:
        print("\nERRO: A variável DATABASE_URL não foi encontrada no seu arquivo .env.")
        print("Certifique-se de que o arquivo está na mesma pasta do seu notebook e que a variável está definida.")
        return

    conn = None
    try:
        print("\nConectando ao banco de dados PostgreSQL no Render...")
        conn = psycopg2.connect(DATABASE_URL)
        print("Conexão bem-sucedida!")

        # Comando SQL para selecionar todos os dados da tabela 'leads_quentes'
        sql_query = "SELECT * FROM leads_quentes ORDER BY data_ingestao DESC;"

        print("Buscando os leads quentes cadastrados...")
        df = pd.read_sql_query(sql_query, conn)

        if df.empty:
            print("\nNenhum lead quente encontrado na tabela ainda.")
        else:
            print(f"\nExibindo {len(df)} leads quentes encontrados:")
            display(df) # Exibe a tabela formatada no Jupyter

    except Exception as e:
        print(f"\nERRO: Não foi possível buscar os dados. Causa: {e}")

    finally:
        if conn:
            conn.close()
            print("\nConexão com o banco de dados foi fechada.")

if __name__ == '__main__':
    visualizar_leads_quentes()


Conectando ao banco de dados PostgreSQL no Render...
Conexão bem-sucedida!
Buscando os leads quentes cadastrados...


C:\Users\Desktop\AppData\Local\Temp\ipykernel_8724\1348347005.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)



Exibindo 130 leads quentes encontrados:


,id,lead_id,nome,email,telefone,empresa,cargo,lead_score,data_ingestao,data_processamento
0,4,NaN,None,None,None,None,None,100,NaT,2025-10-15 21:45:42.395323+00:00
1,6,NaN,None,None,None,None,None,100,NaT,2025-10-15 21:46:15.255576+00:00
2,1,NaN,None,None,None,None,None,100,NaT,2025-10-15 21:42:10.651702+00:00
3,2,NaN,None,None,None,None,None,100,NaT,2025-10-15 21:42:17.906824+00:00
4,3,NaN,None,None,None,None,None,100,NaT,2025-10-15 21:43:29.212109+00:00
...,...,...,...,...,...,...,...,...,...,...
125,17,17.0,Bruno,Gerente@gmail.com,1197887445,Gabriel,Diretor,100,2025-10-15 19:04:43.025000+00:00,2025-10-15 19:11:31.748551+00:00
126,17,17.0,Bruno,Gerente@gmail.com,1197887445,Gabriel,Diretor,100,2025-10-15 19:04:43.025000+00:00,2025-10-15 19:07:47.505874+00:00
127,17,17.0,Bruno,Gerente@gmail.com,1197887445,Gabriel,Diretor,100,2025-10-15 19:04:43.025000+00:00,2025-10-15 19:11:28.301584+00:00
128,3,3.0,Leandro,leandro@teste.com.br,11963195328,Lelis,Gerente,100,2025-10-15 17:31:48.492000+00:00,2025-10-15 17:44:34.917489+00:00



Conexão com o banco de dados foi fechada.


In [26]:
import os
import psycopg2
from dotenv import load_dotenv

# Carrega a DATABASE_URL do seu arquivo .env
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

# Comando SQL para remover a restrição UNIQUE da coluna 'id'
# Primeiro, precisamos descobrir o nome da restrição que o PostgreSQL criou.
# Esta query busca o nome da restrição para a coluna 'id' na tabela 'leads_quentes'.
FIND_CONSTRAINT_NAME_SQL = """
SELECT constraint_name
FROM information_schema.constraint_column_usage
WHERE table_name = 'leads_quentes' AND column_name = 'id' AND table_schema = 'public';
"""

def remover_restricao_unique_id():
    """
    Conecta ao banco de dados e remove a restrição de ID único da tabela 'leads_quentes'.
    """
    conn = None
    if not DATABASE_URL:
        print("ERRO: DATABASE_URL não encontrada no arquivo .env")
        return

    try:
        print("Conectando ao banco de dados...")
        conn = psycopg2.connect(DATABASE_URL)
        cur = conn.cursor()
        print("Conexão bem-sucedida!")

        print("Buscando o nome da restrição UNIQUE para o ID...")
        cur.execute(FIND_CONSTRAINT_NAME_SQL)
        result = cur.fetchone()

        if result:
            constraint_name = result[0]
            print(f"Nome da restrição encontrada: {constraint_name}")
            
            # Agora que temos o nome, podemos removê-la
            alter_table_sql = f"ALTER TABLE leads_quentes DROP CONSTRAINT {constraint_name};"
            print("Executando comando para remover a restrição...")
            cur.execute(alter_table_sql)
            conn.commit()
            print("SUCESSO! A restrição de ID único foi removida. Agora a tabela aceitará IDs duplicados.")
        else:
            print("Aviso: Nenhuma restrição 'UNIQUE' encontrada para a coluna 'id' na tabela 'leads_quentes'.")

    except Exception as e:
        print(f"ERRO: Não foi possível alterar a tabela. Causa: {e}")
        if conn:
            conn.rollback()

    finally:
        if conn:
            cur.close()
            conn.close()
            print("\nConexão com o banco de dados foi fechada.")

if __name__ == '__main__':
    remover_restricao_unique_id()

Conectando ao banco de dados...
Conexão bem-sucedida!
Buscando o nome da restrição UNIQUE para o ID...
Nome da restrição encontrada: leads_quentes_pkey
Executando comando para remover a restrição...
SUCESSO! A restrição de ID único foi removida. Agora a tabela aceitará IDs duplicados.

Conexão com o banco de dados foi fechada.
